In [1]:
import numpy as np

In [21]:
p = [0.2, 0.2, 0.2, 0.2, 0.2]
world = ['green', 'red', 'red', 'green', 'green']
Z = 'red'
pHit = 0.6
pMiss = 0.2

In [13]:
def sense(p, Z):
    prob = np.array(p)
    measure = np.array([pHit if i == Z else pMiss for i in world])
    
    combine_prob = prob*measure
    
    norm_prob = combine_prob/sum(combine_prob)
    return norm_prob

In [11]:
sense(p, Z)

array([0.11111111, 0.33333333, 0.33333333, 0.11111111, 0.11111111])

In [14]:
Z = 'green'

sense(p, Z)

array([0.27272727, 0.09090909, 0.09090909, 0.27272727, 0.27272727])

In [23]:
measurements = ['red', 'green']

for i in measurements:
    p = sense(p, i)
    print("sensing [{}] -> prior distribution {}".format(i, p))

sensing [red] -> prior distribution [0.09090909 0.36363636 0.36363636 0.09090909 0.09090909]
sensing [green] -> prior distribution [0.2 0.2 0.2 0.2 0.2]
